# Preprocessing: Unavailable numerical values (*missing values*)

## Previous steps

In [52]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

housing = pd.read_csv("./data/housing.csv") 
train_set, test_set = train_test_split(housing, test_size=0.2,
    stratify=pd.cut(housing["median_income"], bins=[0., 1.5, 3.0, 4.5, 6., np.inf], labels=[1, 2, 3, 4, 5]),
    random_state=42
    )

**Preprocessing** data is one of the most important tasks in Machine Learning. If the data is not well prepared, *Machine Learning* algorithms won't work correctly. First we'll separate the **predictors** from the **target variable** (the **labels**), since we won't necessarily apply the same **transformations** to both.

In [53]:
X_train = train_set.drop("median_house_value", axis=1) # Remove the dependent variable column
y_train = train_set["median_house_value"].copy() # Save the dependent variable (labels)

X_train.head().T

,12655,15502,2908,14053,20496
longitude,-121.46,-117.23,-119.04,-117.13,-118.7
latitude,38.52,33.09,35.37,32.75,34.28
housing_median_age,29.0,7.0,44.0,24.0,27.0
total_rooms,3873.0,5320.0,1618.0,1877.0,3536.0
total_bedrooms,797.0,855.0,310.0,519.0,646.0
population,2237.0,2015.0,667.0,898.0,1837.0
households,706.0,768.0,300.0,483.0,580.0
median_income,2.1736,6.3373,2.875,2.2264,4.4964
ocean_proximity,INLAND,NEAR OCEAN,INLAND,NEAR OCEAN,<1H OCEAN


## Identification of unavailable values

As we saw at the beginning, the 'total_bedrooms' column has unavailable values. Normally we'll speak of unavailable values, ***missing values***, *null* or ***na* (not available)** as synonyms, although we need to be careful about how those values were collected, since if there are two types of values (null and empty *string*, for example) there could be implicit information.

> **Terminology note:** In data science, several terms are used somewhat interchangeably:
> - **Missing value**: The general concept—a value that should exist but doesn't.
> - **NA (Not Available)**: A common label for missing values, used in R and pandas (`pd.NA`).
> - **NaN (Not a Number)**: Originally from IEEE 754 floating-point standard, used in NumPy/pandas for missing floats.
> - **Null**: Common in databases (SQL NULL); in pandas, `None` for object columns.
> 
> In pandas, `isna()` and `isnull()` are aliases—both detect NA and NaN values. The important distinction is between data that is *structurally* missing (not recorded) vs. *semantically* missing (e.g., "not applicable")—these may require different handling strategies.

In [54]:
X_train.isna().sum() # the isnull() method is an alias for isna()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        158
population              0
households              0
median_income           0
ocean_proximity         0
dtype: int64

In [55]:
null_rows_idx = X_train.isnull().any(axis=1) # indices of rows with null values
X_train.loc[null_rows_idx].head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
1606,-122.08,37.88,26.0,2947.0,NaN,825.0,626.0,2.9330,NEAR BAY
10915,-117.87,33.73,45.0,2264.0,NaN,1970.0,499.0,3.4193,<1H OCEAN
19150,-122.70,38.35,14.0,2313.0,NaN,954.0,397.0,3.7813,<1H OCEAN
4186,-118.23,34.13,48.0,1308.0,NaN,835.0,294.0,4.2891,<1H OCEAN
16885,-122.40,37.58,26.0,3281.0,NaN,1145.0,480.0,6.3580,NEAR OCEAN


## Deletion of rows with null values (***Listwise deletion***)

We can simply delete those incomplete instances, although this is problematic because we're eliminating information. Especially if there are many predictors (since to solve the problem of certain nulls we're losing the information from the other columns).

In [56]:
X_train_ld_tb = X_train.dropna(subset=["total_bedrooms"]) 
X_train_ld_tb.loc[null_rows_idx].head() # verify that rows with null values have been removed

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity


We could also directly delete any row that has a null value in any column:

In [57]:
X_train_ld_any = X_train.dropna(axis=0) # remove rows with null values
X_train_ld_any.loc[null_rows_idx].head() # verify that rows with null values have been removed

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity


## Deletion of the entire column

Deleting the entire column is an option if it's not an important variable, but in this case it seems to be important given that, although that *feature* is not the one that correlates most directly with the target variable, it's one of the two used to calculate `bedrooms_ratio`, which is the second most correlated.

In [58]:
X_train_drop_tb = X_train.drop(columns="total_bedrooms")
X_train_drop_tb.loc[null_rows_idx].head()

,longitude,latitude,housing_median_age,total_rooms,population,households,median_income,ocean_proximity
1606,-122.08,37.88,26.0,2947.0,825.0,626.0,2.9330,NEAR BAY
10915,-117.87,33.73,45.0,2264.0,1970.0,499.0,3.4193,<1H OCEAN
19150,-122.70,38.35,14.0,2313.0,954.0,397.0,3.7813,<1H OCEAN
4186,-118.23,34.13,48.0,1308.0,835.0,294.0,4.2891,<1H OCEAN
16885,-122.40,37.58,26.0,3281.0,1145.0,480.0,6.3580,NEAR OCEAN


The rows are still there in this case because the null indices were searched before. If we search for nulls now in housing_option2, we won't find them.

In [59]:
X_train_drop_tb.isnull().any(axis=None) # verify that there are no null values in the dataset

np.False_

We could also directly delete all columns with nulls:

In [60]:
X_train.dropna(axis=1).isnull().any(axis=None)

np.False_

## Imputation of some value (the median in this case)

**Imputation** of a certain value (such as zero, the mean or the median) to those unavailable fields is an option if we believe that unavailable values don't respond to any specific cause, and don't bias the variable's distribution.

> **Types of missing data:** The appropriate imputation strategy depends on *why* data is missing:
> - **MCAR (Missing Completely At Random)**: The probability of missing is unrelated to any variable. Example: a sensor randomly fails. Simple imputation (mean/median) works well.
> - **MAR (Missing At Random)**: Missingness depends on *observed* variables but not the missing value itself. Example: older survey respondents skip income questions regardless of their actual income. Can use information from other variables to impute.
> - **MNAR (Missing Not At Random)**: Missingness depends on the *unobserved* value itself. Example: high earners deliberately skip income questions. Simple imputation will bias results; requires specialized methods or domain knowledge.
>
> In our dataset, `total_bedrooms` has only 207 missing values (~1%) out of 20,640 records—a small fraction. Without additional information about *why* these values are missing, median imputation is a reasonable choice, as it's robust to potential outliers.

Imputation of the mean (***mean***) is more sensitive to **outliers**, since an extreme value can greatly affect the mean (see [Outliers and capped values](e2e020_eda.ipynb#Outliers-and-capped-values) for outlier handling techniques). The median (***median***) is more robust to extreme values. The mode (***mode***) is the value that repeats most, and is useful for categorical variables, but not as much for continuous variables.

In [61]:
median = X_train["total_bedrooms"].median()
housing_option3 = X_train["total_bedrooms"].fillna(median)
housing_option3.loc[null_rows_idx].head()

1606     433.0
10915    433.0
19150    433.0
4186     433.0
16885    433.0
Name: total_bedrooms, dtype: float64

Now all these rows have in total_bedrooms the median value of total_bedrooms.

The `SimpleImputer` class from scikit-learn allows us to do this more easily. We create an instance of `SimpleImputer` indicating that we want to impute null values with the median, and then use the `fit()` method to calculate the median of each column and the `transform()` method to apply the imputation to all columns.

Let's see how this method would be applied to all numerical fields in the dataframe (remember that 'ocean_proximity' is categorical -text values-, and we can't calculate the median of text).

In [62]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")

In [63]:
housing_num = X_train.select_dtypes(include=[np.number]) # select numerical columns

In [64]:
imputer.fit(housing_num) # calculate the median of each numerical column
imputer.statistics_ # median of each numerical column

array([-118.51   ,   34.26   ,   29.     , 2119.     ,  433.     ,
       1164.     ,  408.     ,    3.54155])

We can verify that the values are the same as those calculated by the dataframe's `median()` method.

In [65]:
housing_num.median().values

array([-118.51   ,   34.26   ,   29.     , 2119.     ,  433.     ,
       1164.     ,  408.     ,    3.54155])

In [66]:
housing_num_array_tr = imputer.transform(housing_num) # replace null values with the median
housing_num_array_tr

array([[-1.2146e+02,  3.8520e+01,  2.9000e+01, ...,  2.2370e+03,
         7.0600e+02,  2.1736e+00],
       [-1.1723e+02,  3.3090e+01,  7.0000e+00, ...,  2.0150e+03,
         7.6800e+02,  6.3373e+00],
       [-1.1904e+02,  3.5370e+01,  4.4000e+01, ...,  6.6700e+02,
         3.0000e+02,  2.8750e+00],
       ...,
       [-1.2272e+02,  3.8440e+01,  4.8000e+01, ...,  4.5800e+02,
         1.7200e+02,  3.1797e+00],
       [-1.2270e+02,  3.8310e+01,  1.4000e+01, ...,  1.2080e+03,
         5.0100e+02,  4.1964e+00],
       [-1.2214e+02,  3.9970e+01,  2.7000e+01, ...,  6.2500e+02,
         1.9700e+02,  3.1319e+00]], shape=(16512, 8))

`transform()` returns a NumPy array, but we could convert it back to a Pandas DataFrame.

In [67]:
housing_tr = pd.DataFrame(housing_num_array_tr, columns=housing_num.columns, index=housing_num.index)
housing_tr.loc[null_rows_idx].head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
1606,-122.08,37.88,26.0,2947.0,433.0,825.0,626.0,2.9330
10915,-117.87,33.73,45.0,2264.0,433.0,1970.0,499.0,3.4193
19150,-122.70,38.35,14.0,2313.0,433.0,954.0,397.0,3.7813
4186,-118.23,34.13,48.0,1308.0,433.0,835.0,294.0,4.2891
16885,-122.40,37.58,26.0,3281.0,433.0,1145.0,480.0,6.3580


We could also directly use the `fit_transform()` method of `SimpleImputer` to calculate the value to impute (with `fit()`) and apply it (with `transform()`) in a single step.

And we could also use the `.set_output(transform="pandas")` method of the imputer so that the result is a Pandas DataFrame.

Therefore, the entire process detailed above can be summarized in a single line of code:

In [68]:
housing_tr = SimpleImputer(strategy="median").set_output(transform="pandas").fit_transform(housing_num)
housing_tr.loc[null_rows_idx].head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
1606,-122.08,37.88,26.0,2947.0,433.0,825.0,626.0,2.9330
10915,-117.87,33.73,45.0,2264.0,433.0,1970.0,499.0,3.4193
19150,-122.70,38.35,14.0,2313.0,433.0,954.0,397.0,3.7813
4186,-118.23,34.13,48.0,1308.0,433.0,835.0,294.0,4.2891
16885,-122.40,37.58,26.0,3281.0,433.0,1145.0,480.0,6.3580


## Predictive models to impute values

There are more advanced methods such as using **prediction models** (treating the column with null values as the target variable and the rest of the columns as *features*). For example, the **K-Nearest Neighbors (KNN)** algorithm could be used to predict the null values of 'total_bedrooms' based on the labeled records. SciKit-Learn has a `KNNImputer` class that does this.